# QuarkNet CRMD Data

The primary reference for the Cosmic Ray Muon Detector (CRMD) data format is the User's Manual located at http://quarknet.i2u2.org/sites/default/files/cf_6000crmdusermanual-small.pdf

The e-Lab analyses begin with what we call "threshold" data stored on our data servers as files with `.thresh` extensions.  The raw data delivered by the CRMD's data acquisition card (DAQ), however, is much different in appearance.  It's instructive to examine to examine this raw data to better understand the more accessible threshold data and how the e-Lab data systems are structured.

### Viewing Cosmic Ray Data

Even if you don't have access to one of QuarkNet's CRMDs, you can still view data uploaded by CRMD operators through the e-Lab interface.  The [Search Data](https://www.i2u2.org/elab/cosmic/data/search.jsp) page allows you to search for cosmic ray data by detector, teacher, or location.  If you don't have a particular dataset in mind, you can search for data from the QuarkNet test detectors under School: Fermilab Test Array.

The search results will show an array of individual data files matching the search criteria, organized by location, detector and date.  You'll see a 'magnifying glass' icon next to each entry that opens a page showing the raw data for this file ([for example](https://www.i2u2.org/elab/cosmic/data/view.jsp?filename=6119.2018.0101.0)).  The "Download" link allows you to download this raw data, explained next, while the "Download Threshold File" link allows you to download the corresponding threshold data, explained later.

## Data Events

The DAQ allows the user to configure the exact conditions - channel coincidence, threshold, etc. - under which an event is recorded as data.  Under typical configurations, a PMT signal in one channel causes the DAQ to open a "gate" for a certain time interval, during which it records all signals on all channels.  If more PMT signals arrive during this gate, their data is recorded as part of the initial event that triggered the gate.  Thus, a single "event" may comprise multiple muon detections and multiple raw data lines.

### Understanding Timing

There are two sources of timing information used in the CRMD: that of the Global Positioning System (GPS), provided by an external receiver, and that of the DAQ's onboard Complex Programmable Logic Device (CPLD) chip.

The **GPS** signal is a UTC time measurement that's the only source of absolute timescale in the CRMD system.  It is precise only to the millisecond, however, which is insufficient to handle the micro- and nanosecond physics of cosmic ray muon events, especially when comparing different DAQs.

The **CPLD** clock operates at 25 MHz, meaning that it increments its reading by one digit every 40ns.  It has a 32-bit storage register typically given in hexadecimal; after counting ticks up to 2<sup>32</sup> = (4,294,967,296)<sub>10</sub> = (FFFFFFFF)<sub>16</sub>, it rolls over to 1 and starts again.  At 40ns per tick, this means that its full time range for one cycle is 171.798691800s, or not quite three minutes.

Though more precise than the GPS clock, the CPLD lacks any absolute time data.  During a standard CRMD data-aquisition run of one or more hours, the CPLD by itself can only record events relative to its current ~3min cycle.  The challenge of the CRMD timing system is in correlating the CRMD to the GPS signal: we use the CPLD to determine the time of a physical event within a ~3min window, in principle to 40ns precision, and then use GPS data to determine the absolute UTC time of this window.  This allows us to record data on cosmic ray muon incidence at the detector with a precision of 10s of nanoseconds.

## I. Raw Data

Raw DAQ output is formatted into lines of 16 "words", the first 10 of which are encoded as hexadecimal numbers.  Below is a sample of five lines of raw data:

```
80EE0049 80 01 00 01 38 01 3C 01 7EB7491F 202133.242 080803 A 04 2 -0389 
80EE004A 24 3D 25 01 00 01 00 01 7EB7491F 202133.242 080803 A 04 2 -0389 
80EE004B 21 01 00 23 00 01 00 01 7EB7491F 202133.242 080803 A 04 2 -0389 
80EE004C 01 2A 00 01 00 01 00 01 7EB7491F 202133.242 080803 A 04 2 -0389 
80EE004D 00 01 00 01 00 39 32 2F 81331170 202133.242 080803 A 04 2 +0610
```

**Trigger count**
The first word is the CPLD trigger count: the CPLD reading at the time of the triggering event.  For example, the five values in the sample data above translate to

`80EE0049` = (`2163081289`)<sub>10</sub> = 86.523251560s

`80EE004A` = (`2163081290`)<sub>10</sub> = 86.523251600s

`80EE004B` = (`2163081291`)<sub>10</sub> = 86.523251640s

`80EE004C` = (`2163081292`)<sub>10</sub> = 86.523251680s

`80EE004D` = (`2163081293`)<sub>10</sub> = 86.523251720s

    * Is it normal for this data to appear in sequential digits like this?  Once triggered, does the DAQ simply record all four channels for every CPLD cycle up to the gate width?
    * For a single event represented by multiple (here, 5) lines of data, why does each line have a different "trigger count" clock reading?  Does a single event not correspond to a single trigger/gate?
    * What event does this time reading represent?  The opening of the trigger gate? The last 1PPS GPS pulse?

**Pulse edges**
The next eight words are two-digit hex numbers that parameterize the **rising edges** and **falling edges** of the PMT signal from each of the four input channels, and they share similar formats.  The eight words encode information for, in order,

**RE0**, the rising edge on channel 0

**FE0**, the falling edge on channel 0

**RE1**, the rising edge on channel 1

**FE1**, the falling edge on channel 1

**RE2**, the rising edge on channel 2

**FE2**, the falling edge on channel 2

**RE3**, the rising edge on channel 3

**FE3**, the falling edge on channel 3

The encoding scheme of these words is more clearly understood using their eight-digit binary representation, for example (`3c`)<sub>hex</sub> = (`00111100`)<sub>bin</sub>.  

In binary, counting from the rightmost (i.e., lowest) digits, the first five bits represent the TMC clock count of the time that the edge crosses the threshold voltage.  The TMC is the "Time Measurement Chip", a type of Time-to-Digital Converter (TDC) that's integrated into the DAQ board, one for each channel.

The sixth bit is the channel edge tag, with `1` representing a valid rising or falling edge and `0` representing no valid edge. 

The seventh bit is not used and is always zero.  

The eighth (leftmost) bit is the trigger tag, with `1` indicating that this is the triggering signal of a new event and `0` indicating that this is a follow-up signal to an event whose trigger was recorded in a previous line.

Only rising-edge signals use the `new trigger` bit; for falling-edge signals both of the two leftmost binary digits are unused and always zero.  Other than this, the encoding is the same for both types.

For example, we can interpret the rising/falling edge values of first line of the sample data as

**RE0:** (`80`)<sub>hex</sub> = (`10000000`)<sub>bin</sub> = (`1`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`1`)<sub>edge</sub>(`00000`)<sub>TMC</sub>

**FE0:** (`01`)<sub>hex</sub> = (`00000001`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`0`)<sub>edge</sub>(`00001`)<sub>TMC</sub>

**RE1:** (`00`)<sub>hex</sub> = (`00000000`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`0`)<sub>edge</sub>(`00000`)<sub>TMC</sub>

**FE1:** (`01`)<sub>hex</sub> = (`00000001`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`0`)<sub>edge</sub>(`00001`)<sub>TMC</sub>

**RE2:** (`38`)<sub>hex</sub> = (`00111000`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`1`)<sub>edge</sub>(`11000`)<sub>TMC</sub>

**FE2:** (`01`)<sub>hex</sub> = (`00000001`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`0`)<sub>edge</sub>(`00001`)<sub>TMC</sub>

**RE3:** (`3C`)<sub>hex</sub> = (`00111100`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`1`)<sub>edge</sub>(`11100`)<sub>TMC</sub>

**FE3:** (`01`)<sub>hex</sub> = (`00000001`)<sub>bin</sub> = (`0`)<sub>trigger</sub>(`0`)<sub>unused</sub>(`0`)<sub>edge</sub>(`00001`)<sub>TMC</sub>

    * Why are there no valid falling edges in this line of data?


The TMC clock itself    



    
* (`24`)<sub>hex</sub> = (`00100100`)<sub>bin</sub>
* (`21`)<sub>hex</sub> = (`00100001`)<sub>bin</sub>
* (`01`)<sub>hex</sub> = (`00100001`)<sub>bin</sub>
* (`00`)<sub>hex</sub> = (`00000000`)<sub>bin</sub>


The tenth word in each line of data is the CPLD clock reading of when the most recent 1PPS signal was received.  Its values are interpreted the same way as the first word.  For example, the values shown in the five lines of sample data are

`7EB7491F` = (`2125941023`)<sub>dec</sub> = 85.037640920s

`81331170` = (`2167607664`)<sub>dec</sub> = 86.704306560s

    * It seems noteworthy that the 5th line has a "most recent" 1PPS reading of later than the first-word reading in any of the lines.

## II. Threshold data

When raw data is uploaded to the e-Lab servers, lines recording user command inputs (that is, non-data lines) are stripped, data spanning multiple days are split into individual days, and the resulting data is then processed into *threshold files* (`.thresh`).  Threshold files record individual PMT signals as one signal per line: DAQ and channel, date, rising edge time, falling edge time, and time over threshold.

Here's an example of threshold data from the Fermilab Test Array (DAQ 6119) taken Monday January 1, 2018:

In [3]:
!head -10 ./files/6119.2018.0101.0.thresh

#$md5
#md5_hex(0)
#ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec), RISING EDGE(INT), FALLING EDGE(INT)
6119.1	2458119	0.5000008193406684	0.5000008193410012	28.75	4320007079103375	4320007079106250
6119.3	2458119	0.5000008193406540	0.5000008193410446	33.75	4320007079103250	4320007079106626
6119.4	2458119	0.5000009625294560	0.5000009625298176	31.24	4320008316254500	4320008316257624
6119.2	2458119	0.5000009625294126	0.5000009625299044	42.49	4320008316254126	4320008316258374
6119.2	2458119	0.5000043927394676	0.5000043927398582	33.75	4320037953269000	4320037953272374
6119.4	2458119	0.5000043927394966	0.5000043927398293	28.75	4320037953269250	4320037953272125
6119.1	2458119	0.5000049710932147	0.5000049710935763	31.24	4320042950245375	4320042950248500


Despite the header, the `RISING EDGE(sec)` and `FALLING EDGE(sec)` values are in days, as measured from the most recent UTC 12:00 event.  The least significant bit of these values, $10^{-16}$ days, is equal to about $8.64\times 10^{-12}$s, or nearly a hundredth of a nanosecond, and much less than the expected precision of DAQ timing.  For comparison, 10ns, the precision the DAQ timing system aims for, is about $1.16\times 10^{-13}$ days, so you'll typically only need to pay attention to the last four or five digits of the `RISING/FALLING EDGE(sec)` values.

The `TIME OVER THRESHOLD` column is the difference between the two previous `RISING EDGE` and `FALLING EDGE` values, given in nanoseconds and rounded to the nearest hundredth of a nanosecond.  The `RISING EDGE(INT)` and `FALLING EDGE(INT)` values are the respective `RISING EDGE(sec)` and `FALLING EDGE(sec)` values converted to hundredths of a nanosecond and rounded to the nearest integer in those units.  Like the first pair of values, the second pair are also measured from the most recent UTC 12:00 event.  Recall that when multi-day data is uploaded, the e-Labs split the data into days at UTC 00:00, which is exactly half a day from UTC 12:00 -- that's why you'll notice that many cosmic ray threshold files start with `RISING/FALLING EDGE(sec)` values that are just greater than 0.5 days.